In [ ]:
import sys
import os
project_root = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, project_root)
#импорты загрузки 

import InterfaceWidgets.data_load_widgets as data_load_widgets
import InterfaceWidgets.column_selection_widgets as column_selection_widgets

#импорты EDA
import InterfaceWidgets.EDA_widgets as eda_widgets

#импорты экспериментального ядра
import ExperimentsCore.experiments_core as exp_core
import InterfaceWidgets.experiment_config_widgets as exp_config_widgets
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import pandas as pd

# Создаем виджет Output для вывода информации о выбранной колонке
column_info_output = widgets.Output()

# Создаем виджет Output для информации о создании распределения
distribution_info_output = widgets.Output()

# Переменная для хранения статистик распределения
current_statistics = None

# Функция для расчета статистик распределения
def calculate_distribution_statistics(rv, df_dist):
    """
    Рассчитывает различные статистики для распределения.
    
    Args:
        rv: Объект дискретного распределения
        df_dist: DataFrame с значениями и вероятностями
        
    Returns:
        dict: Словарь с рассчитанными статистиками
    """
    stats = {}
    
    # Рассчитываем среднее
    if hasattr(rv, 'mean'):
        stats['mean'] = rv.mean()
    else:
        # Если у rv нет метода mean(), вычисляем вручную
        xk = df_dist['value'].values
        pk = df_dist['probability'].values
        stats['mean'] = np.sum(xk * pk)
    
    # Рассчитываем медиану (как квантиль 0.5)
    # Для дискретного распределения найдем значение, где CDF >= 0.5
    try:
        median_idx = np.where(df_dist['cumulative_probability'] >= 0.5)[0][0]
        stats['median'] = df_dist['value'].iloc[median_idx]
    except (IndexError, KeyError):
        stats['median'] = None
    
    # В будущем здесь можно добавить расчет других статистик
    # stats['quantile'] = ...
    # stats['proportion'] = ...
    
    return stats

# Обработчик создания распределения после EDA
def handle_distribution_creation(series):
    global current_statistics
    with distribution_info_output:
        distribution_info_output.clear_output(wait=True)
        print("Создание дискретного распределения...")
        rv, df_dist, round_digits = exp_core.discrete_dist_creation(series)
        
        # Сохраняем статистики для использования в интерфейсе конфигурации
        current_statistics = calculate_distribution_statistics(rv, df_dist)
            
        display(rv, df_dist, round_digits)
    
    # Отображаем интерфейс конфигурации с передачей статистик распределения
    exp_config_widgets.display_experiment_config_interface(current_statistics=current_statistics)

# Обработчик выбора колонки
def handle_column_selection(selected_column, data_dict):
    with column_info_output:
        column_info_output.clear_output(wait=True)
        print(f"Выбрана колонка: {selected_column}")
        print(f"Получен словарь данных с {len(data_dict['df'])} строками")
    # Запускаем EDA анализ для выбранной колонки с передачей callback'а для создания распределения
    series = data_dict['df'][selected_column]
    eda_widgets.run_eda_analysis(selected_column, data_dict, 
                                on_eda_complete=lambda: handle_distribution_creation(series))

# Обработчик загрузки данных
def handle_data_loaded(data_dict):
    # Когда данные загружены, показываем селектор колонок
    column_selection_widgets.column_selection_widgets(
        data_dict, 
        on_column_selected=handle_column_selection
    )
    # Отображаем виджет для информации о выбранной колонке после селектора
    display(column_info_output)
    # Виджет для информации о распределении отобразится после графиков в функции run_eda_analysis

# Показываем интерфейс загрузки с callback'ом
data_load_widgets.display_interface(on_data_loaded=handle_data_loaded)